1.Membaca dataset

In [86]:
df = pd.read_csv('/content/data-manual.csv', engine='python', on_bad_lines='skip')
df.head()


,Review,Label
0,"Makanan laut segar, harga terjangkau, cocok un...",Positif
1,Langsung di sambut dengan ramah n megah,NaN
2,Salah satu tempat makan seafood di seputaran G...,NaN
3,"Lokasi strategis, parkir mobil cuma bisa di te...",NaN
4,"Enak, ga terlalu pricey, worth it",NaN


2. Cek kolom dataset

In [87]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1103 non-null   object
 1   Label   298 non-null    object
dtypes: object(2)
memory usage: 17.4+ KB


,Review,Label
0,"Makanan laut segar, harga terjangkau, cocok un...",Positif
1,Langsung di sambut dengan ramah n megah,NaN
2,Salah satu tempat makan seafood di seputaran G...,NaN
3,"Lokasi strategis, parkir mobil cuma bisa di te...",NaN
4,"Enak, ga terlalu pricey, worth it",NaN


3. Bersihkan data kosong

In [88]:
df = df.dropna(subset=['Review'])
df = df.reset_index(drop=True)
df.head()


,Review,Label
0,"Makanan laut segar, harga terjangkau, cocok un...",Positif
1,Langsung di sambut dengan ramah n megah,NaN
2,Salah satu tempat makan seafood di seputaran G...,NaN
3,"Lokasi strategis, parkir mobil cuma bisa di te...",NaN
4,"Enak, ga terlalu pricey, worth it",NaN


4. Normalisasi Label (Pastikan konsisten)

In [89]:
df['Label'] = df['Label'].str.lower()

df['Label'] = df['Label'].replace({
    'positif': 'positif',
    'positive': 'positif',
    'positif ': 'positif',
    'positif  ': 'positif',
    'posisitif': 'positif',
    'negatif': 'negatif',
    'negative': 'negatif'
})

df['Label'].unique()


array(['positif', nan, 'negatif'], dtype=object)

5. Cek jumlah label

In [90]:
df['Label'].value_counts()


,count
Label,
negatif,150
positif,148


6.Pisahkan 300 data berlabel & 700 data tidak berlabel

In [91]:
# data berlabel (manual 300)
df_labeled = df.dropna(subset=['Label']).reset_index(drop=True)

# data tidak berlabel (700)
df_unlabeled = df[df['Label'].isna()].reset_index(drop=True)

print("Jumlah data berlabel:", len(df_labeled))
print("Jumlah data tidak berlabel:", len(df_unlabeled))


Jumlah data berlabel: 298
Jumlah data tidak berlabel: 805


#baseline Model Naive Bayes

SPlit DAta

In [92]:
X = df_labeled['Review']
y = df_labeled['Label']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
TF-IDF

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase=True)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


Latih model Naive Bayes

In [94]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_tfidf, y_train)


MultinomialNB()

Evaluasi

In [95]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test_tfidf)

print("Akurasi :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Akurasi : 0.6666666666666666
              precision    recall  f1-score   support

     negatif       0.65      0.73      0.69        30
     positif       0.69      0.60      0.64        30

    accuracy                           0.67        60
   macro avg       0.67      0.67      0.67        60
weighted avg       0.67      0.67      0.67        60



In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase=True)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.shape, X_test_tfidf.shape


((238, 1199), (60, 1199))

Ambil data yang belum berlabel

In [97]:
df_unlabeled


,Review,Label
0,Langsung di sambut dengan ramah n megah,NaN
1,Salah satu tempat makan seafood di seputaran G...,NaN
2,"Lokasi strategis, parkir mobil cuma bisa di te...",NaN
3,"Enak, ga terlalu pricey, worth it",NaN
4,"enak, lumayan murah karena porsinya besar. keh...",NaN
...,...,...
800,"Top semua dehh 😁\nRasa, pelayanan, kecepatan p...",NaN
801,"Karyawan nya ramah"", pelayanan cepat, untuk ha...",NaN
802,Tidak pernah gagal.. Suka semua yang ada telur...,NaN
803,Murah dan enak,NaN


Ubah data jadi TF-IDF

In [98]:
df_unlabeled = df[df['Label'].isna()].reset_index(drop=True)
len(df_unlabeled)


805

In [99]:
X_unlabeled_tfidf = tfidf.transform(df_unlabeled['Review'])


Prediksi label dari model Naive Bayes

In [101]:
pseudo_label = model.predict(X_unlabeled_tfidf)
pseudo_prob = model.predict_proba(X_unlabeled_tfidf).max(axis=1)


Gabungkan hasil ke dataframe

In [102]:
df_unlabeled['Label_model'] = pseudo_label
df_unlabeled['Prob_model'] = pseudo_prob

df_unlabeled.head()


,Review,Label,Label_model,Prob_model
0,Langsung di sambut dengan ramah n megah,NaN,positif,0.563012
1,Salah satu tempat makan seafood di seputaran G...,NaN,positif,0.506823
2,"Lokasi strategis, parkir mobil cuma bisa di te...",NaN,negatif,0.651890
3,"Enak, ga terlalu pricey, worth it",NaN,negatif,0.630983
4,"enak, lumayan murah karena porsinya besar. keh...",NaN,positif,0.752659


Simpan hasil pseudo-label

In [103]:
df_unlabeled.to_csv("self_training_model.csv", index=False)


In [104]:
df_reviews_only = df_unlabeled[['Review']].copy()

df_reviews_only.to_csv("reviews_for_llm.csv", index=False)
df_reviews_only.head()


,Review
0,Langsung di sambut dengan ramah n megah
1,Salah satu tempat makan seafood di seputaran G...
2,"Lokasi strategis, parkir mobil cuma bisa di te..."
3,"Enak, ga terlalu pricey, worth it"
4,"enak, lumayan murah karena porsinya besar. keh..."


In [105]:
from google.colab import files
files.download("reviews_for_llm.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

labeli otomatis semuanya dengan LLM.

In [106]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

# 1. Load Data
# The previous error occurred because 'reviews_for_llm.csv' only contained 'Review' column.
# We will now load 'self_training_model.csv' which contains 'Review', original 'Label' (with NaNs for unlabeled),
# and the 'Label_model' (pseudo-labels generated by the Naive Bayes model) and 'Prob_model'.
df_raw = pd.read_csv('self_training_model.csv')

# 2. Preprocessing Ringan
# Ensure 'Review' column is string and handle missing values
df_raw['Review'] = df_raw['Review'].fillna('').astype(str)

# The goal of this block is to train a new model.
# We have original labels in `df_raw['Label']` (some NaN) and pseudo-labels in `df_raw['Label_model']`.
# Let's create a combined 'Label' column for training.
# Use original 'Label' if available, otherwise use 'Label_model'.
df_raw['Final_Label'] = df_raw['Label'].fillna(df_raw['Label_model'])

# Normalize labels to Title Case for consistency (e.g., "Positif", "Negatif")
df_raw['Final_Label'] = df_raw['Final_Label'].astype(str).str.title()

# 3. Prepare Data for Training
# In this modified approach, 'train_df' will contain all reviews with a 'Final_Label'.
# 'predict_df' would only contain entries where 'Final_Label' is still NaN,
# which shouldn't happen if we've filled NaNs with 'Label_model'.
# So, 'predict_df' will likely be empty.
train_df = df_raw.copy()
train_df['Label'] = train_df['Final_Label'] # Assign the combined label as 'Label' for training

# Filter for valid labels to ensure quality of training data
valid_labels = ['Positif', 'Negatif']
train_df = train_df[train_df['Label'].isin(valid_labels)].copy() # Use .copy() to avoid SettingWithCopyWarning

# Create an empty predict_df as all data is now effectively labeled for this step
predict_df = pd.DataFrame(columns=train_df.columns)

if len(train_df) > 0:
    # 4. Train Model
    # Using LinearSVC for text classification
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=2, ngram_range=(1, 2))), # Unigram & Bigram (e.g., "tidak enak")
        ('clf', LinearSVC(random_state=42, dual='auto'))
    ])

    pipeline.fit(train_df['Review'], train_df['Label'])

    # 5. Predict Data Kosong (this part will effectively be skipped if predict_df is empty)
    if len(predict_df) > 0:
        predicted_labels = pipeline.predict(predict_df['Review'])
        predict_df['Label'] = predicted_labels
else:
    # Fallback if no valid training data
    print("Warning: No valid training data found. Cannot train the model.")
    # If predict_df was intended for new unlabeled data, a different handling would be needed.

# 6. Combine Results
# Since `predict_df` is empty, `final_df` will effectively be the `train_df` (which is fully labeled).
final_df = train_df.copy()

# 7. Format Output Sesuai Request (Review, Label_LLM)
output_df = pd.DataFrame({
    'Review': final_df['Review'],
    'Label_LLM': final_df['Label'] # This column now contains original or pseudo-labels
})

# Save to CSV
output_filename = 'reviews_processed.csv'
output_df.to_csv(output_filename, index=False)

output_filename

'reviews_processed.csv'